پیاده سازی مقاله deeplearning hand written recognition مجتبی رمضانی

در این مرحله پکیج ها بارگزاری می شوند پکیج های استفاده شده در پروژه شامل 


---
tensorflow,
numpy,
pandas,
multiprocessing,
time 




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
from multiprocessing import Process,Manager
import time
graph=tf.Graph()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


در این مرحله مدل CNN با وزن قبلی هت شناسایی اعداد ایجاد می گردد

In [ ]:

start_time = time.time()
def CNN_Part(number,training_data,training_labels,validation_data,return_dict):

        print ("start")
        num_lables=10
        image_size=28
        
        learning_rate=1e-2
        batch_size=50

        def weights(shape):
            return tf.Variable(tf.truncated_normal(shape,stddev=0.1))
        
        def biases(shape):
            return tf.Variable(tf.constant(0.1,shape=shape))
        
        def conv2d(data,wts):
            return tf.nn.conv2d(data,wts,strides=[1,1,1,1],padding='SAME',use_cudnn_on_gpu=False)
        
        def max_pooling(value):
            return tf.nn.max_pool(value,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
        
        def accuracy(predictions, labels):
          return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
                  / predictions.shape[0])
        
        
        
        
        global graph
        graph=tf.Graph()
        with graph.as_default():
        
        
            tf_train_data=tf.placeholder(tf.float32,shape=[batch_size,image_size*image_size])
            tf_train_labels=tf.placeholder(tf.float32,shape=[batch_size,num_lables])
        
            tf_valid_data=tf.constant(validation_data)
            tf_test_data=tf.placeholder(tf.float32,shape=[batch_size,image_size*image_size])
            
            
            layer1_w=weights([5,5,1,32])
            layer1_b=biases([32])
        
            layer2_w=weights([5,5,32,64])
            layer2_b=biases([64])
        
            fully_w=weights([7*7*64,1024])
            fully_b=biases([1024])
            final_w=weights([1024,num_lables])
            final_b=biases([num_lables])
        
        
            def model(data):
        
     
                image=tf.reshape(data,[-1,image_size,image_size,1])
        
                hidden1_conv=tf.nn.relu(conv2d(image,layer1_w)+layer1_b)
                hidden1_pool=max_pooling(hidden1_conv)
        
                hidden2_conv=tf.nn.relu(conv2d(hidden1_pool,layer2_w)+layer2_b)
                hidden2_pool=max_pooling(hidden2_conv)
        
                hidden2_pool_flat=tf.reshape(hidden2_pool,[-1,7*7*64])
                hidden_fully=tf.nn.relu(tf.matmul(hidden2_pool_flat,fully_w)+fully_b)
                
                hidden_final=tf.nn.dropout(hidden_fully,rate=0.1)
                
                
                return  tf.matmul(hidden_final,final_w)+final_b
        
            print ("start2")
            logits=model(tf_train_data)
            
            loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=tf_train_labels))

            optimization=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
            
            valid_predections=tf.nn.softmax(model(tf_valid_data))
            
            train_predections=tf.nn.softmax(logits)            
            valid_parameter=model(tf_valid_data)
            
            
           
    
        num_steps=100
        
               
        with tf.Session(graph=graph) as session:
            
        
            tf.initialize_all_variables().run()
            print('initialized')
        
            counter=0
            for step in range(num_steps):
                if counter>len(training_data):
                    prem=np.arange(len(training_data))
                    np.random.shuffle(prem)
                    training_data=training_data[prem]
                    training_labels=training_labels[prem]
                    counter=0
                offset = (step * batch_size) % (training_labels.shape[0] - batch_size)
                batch_data = training_data[offset:(offset + batch_size), :]
                batch_labels = training_labels[offset:(offset + batch_size), :]
        
                feed_dict={tf_train_data:batch_data,tf_train_labels:batch_labels}
                
                print ("Step",counter)
                _,parameter= session.run([optimization,valid_parameter], feed_dict=feed_dict)
                

                counter+=1

        return_dict[number]=parameter
        print (parameter)
        
        return parameter





def splitdata(data):

        data1=data[1:2000]
        data2=data[10000:20000]
        data3=data[20000:30000]
        
        images1=np.array(data1.drop('label',axis=1)).astype(np.float32)
        images1=np.multiply(images1,1.0/255.0)
        
        labels1=np.array(data1.label).astype(np.float32)        
        labels1=(np.arange(10)==labels1[:,None]).astype(np.float32)              
        
        images2=np.array(data2.drop('label',axis=1)).astype(np.float32)
        images2=np.multiply(images2,1.0/255.0)
        
        labels2=np.array(data2.label).astype(np.float32)        
        labels2=(np.arange(10)==labels2[:,None]).astype(np.float32)
                       
        images3=np.array(data3.drop('label',axis=1)).astype(np.float32)
        images3=np.multiply(images3,1.0/255.0)
        labels3=np.array(data3.label).astype(np.float32)        
        labels3=(np.arange(10)==labels3[:,None]).astype(np.float32)
       
        training_data1=images2
        training_labels1=labels2
        
        training_data2=images3
        training_labels2=labels3
                        
        validation_data=images1
        validation_labels=labels1
        
        
        return (training_data1,training_labels1,training_data2,training_labels2,validation_data,validation_labels)


def Caculate_P(someweight,validation_labels):
        def accuracy(predictions, labels):
          return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
                  / predictions.shape[0])
        
        with tf.Session(graph=graph) as session:
            
            position=tf.nn.softmax(someweight)
            
            position=position.eval()
                
            print (position )     
                            
            print('Validation accuracy: %.1f%%' % accuracy(position, validation_labels))              
            
            return position


اجرا بدون معماری mapreduce

In [ ]:

if __name__ == '__main__': 
     time0=time.clock()
     print (time0)
     manager = Manager()
     return_dict = manager.dict()
     jobs = []
    
     data=pd.read_csv('drive/My Drive/train_test.csv',header=0)
     
     
     dataset=splitdata(data)    
     CNN_Part(1,dataset[0],dataset[1],dataset[4],return_dict)
     CNN_Part(2,dataset[2],dataset[3],dataset[4],return_dict)

     finalpare=return_dict.get(1)+return_dict.get(2)
     Caculate_P(finalpare,dataset[5])
     save=time.time() - start_time
      
     print ("Total Time:", save)
 




111.874999


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until


start
start2
initialized
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Step 8
Step 9
Step 10
Step 11
Step 12
Step 13
Step 14
Step 15
Step 16
Step 17
Step 18
Step 19
Step 20
Step 21
Step 22
Step 23
Step 24
Step 25
Step 26
Step 27
Step 28
Step 29
Step 30
Step 31
Step 32
Step 33
Step 34
Step 35
Step 36
Step 37
Step 38
Step 39
Step 40
Step 41
Step 42
Step 43
Step 44
Step 45
Step 46
Step 47
Step 48
Step 49
Step 50
Step 51
Step 52
Step 53
Step 54
Step 55
Step 56
Step 57
Step 58
Step 59
Step 60
Step 61
Step 62
Step 63
Step 64
Step 65
Step 66
Step 67
Step 68
Step 69
Step 70
Step 71
Step 72
Step 73
Step 74
Step 75
Step 76
Step 77
Step 78
Step 79
Step 80
Step 81
Step 82
Step 83
Step 84
Step 85
Step 86
Step 87
Step 88
Step 89
Step 90
Step 91
Step 92
Step 93
Step 94
Step 95
Step 96
Step 97
Step 98
Step 99
[[ 7.736147   -7.8991027   2.2345102  ... -4.442497    1.3243134
  -3.6283865 ]
 [-2.1881347   7.5436945  -2.3315513  ... -1.1239115   0.52868915
  -2.6776295 ]
 [-0.62171304 -4.336617 

اجرا با معماری mapreduce

In [ ]:
if __name__ == '__main__': 
     time0=time.clock()
     print (time0)
     manager = Manager()
     return_dict = manager.dict()
     jobs = []
    
     data=pd.read_csv('drive/My Drive/train_test.csv',header=0)
     
     dataset=splitdata(data)
     
     batch1=Process(target=CNN_Part,args=(1,dataset[0],dataset[1],dataset[4],return_dict))
     jobs.append(batch1)
     batch1.start()
    
     batch2=Process(target=CNN_Part,args=(2,dataset[2],dataset[3],dataset[4],return_dict)) 
     jobs.append(batch2)
     batch2.start()
 
     for proc in jobs:
        proc.join()
      

     finalpare=return_dict.get(1)+return_dict.get(2)

     Caculate_P(finalpare,dataset[5])
     
     save=time.time() - start_time
     

     print ("Total Time:", save)
 

666.829977


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


start
start
start2
start2
